<a href="https://colab.research.google.com/github/suhas919/LVQExperiments/blob/main/LVQ_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [4]:
import numpy as np
from queue import PriorityQueue
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

dataset = 'NONE'

Mounted at /content/drive


In [5]:
# Load GIST CSV
# dataset = 'GIST'

# Load DBPEDIA Entities Dataset obtained from huggingface
# dataset = 'DBPEDIA'

# Load GLOVE CSV
dataset = 'GLOVE'

if dataset == 'GIST':
  data_train = np.loadtxt('/content/drive/MyDrive/ScalarQuantization/GIST/gist_train_100K.csv', delimiter=',')
  print(np.shape(data_train))
  print(data_train)

  data_test = np.loadtxt('/content/drive/MyDrive/ScalarQuantization/GIST/gist_test.csv', delimiter=',')
  print(np.shape(data_test))
  print(data_test)

elif dataset == 'DBPEDIA':
  df1 = pd.read_parquet('/content/drive/MyDrive/ScalarQuantization/dbpedia-entities-openai3-text-embedding-3-small-1536-100K/data/train-00000-of-00004.parquet')
  df2 = pd.read_parquet('/content/drive/MyDrive/ScalarQuantization/dbpedia-entities-openai3-text-embedding-3-small-1536-100K/data/train-00001-of-00004.parquet')
  df3 = pd.read_parquet('/content/drive/MyDrive/ScalarQuantization/dbpedia-entities-openai3-text-embedding-3-small-1536-100K/data/train-00002-of-00004.parquet')
  df4 = pd.read_parquet('/content/drive/MyDrive/ScalarQuantization/dbpedia-entities-openai3-text-embedding-3-small-1536-100K/data/train-00003-of-00004.parquet')
  df = pd.concat([df1, df2, df3, df4], axis=0)
  df.reset_index(drop=True, inplace=True)
  array = df['text-embedding-3-small-1536-embedding'].to_numpy()
  array = np.vstack(array)
  print(array.shape)
  print(array)

elif dataset == 'GLOVE':
  data_train = np.loadtxt('/content/drive/MyDrive/ScalarQuantization/Glove/glove200_train_100k.csv', delimiter=',')
  print(np.shape(data_train))
  print(data_train)

  data_test = np.loadtxt('/content/drive/MyDrive/ScalarQuantization/Glove/glove200_test.csv', delimiter=',')
  print(np.shape(data_test))
  print(data_test)

(100000, 200)
[[-0.23010001  0.07255     0.24698    ... -0.29379001 -0.20446999
  -0.23791   ]
 [-0.10979    -1.07290006  0.69889998 ... -0.21582    -0.48061001
  -0.11557   ]
 [ 0.075252    0.59464002  0.24169999 ... -0.078789    1.01730001
  -0.26394001]
 ...
 [ 0.51151001 -0.54062003 -0.1815     ...  0.59437001  0.5273
  -0.38343   ]
 [-0.22105999  0.12649     0.40748999 ...  0.31575    -0.17274
  -0.75369   ]
 [ 0.045832   -0.61195999  0.25566    ...  0.41709    -0.48008999
  -0.75964999]]
(10000, 200)
[[-0.14205     0.40746999 -0.37594    ...  0.22135    -0.49772
  -0.45609   ]
 [-0.041663   -0.22078     0.37573001 ... -0.19888    -0.53834999
   0.30063   ]
 [ 0.25580001 -0.11698    -0.32681999 ...  0.37365001 -0.79307997
  -0.066078  ]
 ...
 [ 0.23751999  0.96261001  0.15023001 ...  0.32071999 -0.39925
  -0.46586999]
 [ 0.40832001 -0.29617     0.70454001 ...  0.81573999 -0.16338
   0.65101999]
 [-0.53675997 -0.77855003  0.71265    ...  0.33496001  0.44966
  -0.92355001]]


In [6]:
# Setup and Hyperparameters

K = 10                 # For top-"K"

# distfunc = 'EUCLIDEAN'  # distance function: EUCLIDEAN or COSINE
distfunc = 'COSINE'

# qdtype = 'UB1'          # Quantized data type: UB1 or SB1
qdtype = 'SB1'

alg = 'PDN'     # Algorithm type: LVQ or PDN

num_queries = 500

B1 = 8  # Number of bits for quantization
res_factors = [1, 10, 100]  # Rescoring factors

# Sample Xin
# Xin = np.array([
#         [1.2, 3.5, 5.1, 7.3],
#         [2.1, 4.5, 6.1, 8.3],
#         [1.8, 3.0, 4.8, 7.0],
#         [2.0, 4.0, 5.0, 8.0],
#         [1.5, 3.2, 5.3, 7.8],
#         [2.2, 4.3, 6.5, 8.6],
#         [1.1, 3.4, 5.6, 7.4],
#         [2.5, 4.8, 6.3, 8.9],
#         [1.9, 3.9, 5.2, 7.7],
#         [2.0, 4.2, 6.0, 8.1],
#     ])
if dataset == 'GIST':
  Xin = data_train
  # Xin = data_train[:10000, :]
elif dataset == 'DBPEDIA':
  Xin = array[:99000,:]
  Xin = array[:99000,:]
elif dataset == 'GLOVE':
  Xin = data_train[:100000,:]
print("Original Vectors:")
print(np.shape(Xin))
print(Xin)

assert(K <= np.shape(Xin)[0])

if dataset == 'GIST':
  Xquery = data_test[:num_queries, :]
elif dataset == 'DBPEDIA':
  Xquery = array[(100000 - num_queries):, :]
elif dataset == 'GLOVE':
  Xquery = data_test[:num_queries, :]
print("Query Vectors:")
print(np.shape(Xquery))
print(Xquery)

Original Vectors:
(100000, 200)
[[-0.23010001  0.07255     0.24698    ... -0.29379001 -0.20446999
  -0.23791   ]
 [-0.10979    -1.07290006  0.69889998 ... -0.21582    -0.48061001
  -0.11557   ]
 [ 0.075252    0.59464002  0.24169999 ... -0.078789    1.01730001
  -0.26394001]
 ...
 [ 0.51151001 -0.54062003 -0.1815     ...  0.59437001  0.5273
  -0.38343   ]
 [-0.22105999  0.12649     0.40748999 ...  0.31575    -0.17274
  -0.75369   ]
 [ 0.045832   -0.61195999  0.25566    ...  0.41709    -0.48008999
  -0.75964999]]
Query Vectors:
(500, 200)
[[-0.14205     0.40746999 -0.37594    ...  0.22135    -0.49772
  -0.45609   ]
 [-0.041663   -0.22078     0.37573001 ... -0.19888    -0.53834999
   0.30063   ]
 [ 0.25580001 -0.11698    -0.32681999 ...  0.37365001 -0.79307997
  -0.066078  ]
 ...
 [-0.46079999 -0.095973   -0.11125    ...  0.071297   -1.12960005
   0.50244999]
 [-0.27873999 -0.22116999  0.70613003 ... -0.34167999 -0.17739999
  -0.12989999]
 [-0.1049     -0.12008     0.35413    ...  0.84759

In [7]:
class LVQFirstLevel:
    distfunc = 'EUCLIDEAN'

    def __init__(self, distfunc):
      self.distfunc = distfunc

    ## LVQ_first_level

    # "Quantized Data Type"
    # qdtype: 'UB1' -> {0...255}
    # qdtype: 'SB1' -> {-128...127}

    # "Algorithm"
    # alg: 'LVQ' -> LVQ implementation, Per Vector Norm.
    #       l, u and delta are computed along the vector for each vector
    # alg: 'PDN' -> Per Dimension Normalization.
    #       l, u and delta are computed across vectors for each dimension
    # alg: 'GMM' -> Global Min Max Algorithm
    #       l, u and delta are computed across vectors and across all dimensions

    # isQuery: False indicates these are graph vectors, True indicates these are query vectors
    @staticmethod
    def LVQ_first_level(Xin, B1, alg = 'LVQ', isQuery = False, qdtype = 'UB1', X_means_p = None, l_p = None, delta_p = None):
        if (alg not in ['LVQ', 'PDN', 'GMM']):
          assert False, "Unrecognized algorithm parameter"
        if (qdtype not in ['UB1', 'SB1']):
          assert False, "Unrecognized quantization data type parameter"

        if(isQuery):
          if(alg == 'LVQ'):
            assert(X_means_p is not None)
          elif(alg == 'PDN' or alg == 'GMM'):
            assert(l_p is not None)
            assert(delta_p is not None)
          # Todo: Check shapes of arguments

        print("Xin shape:", np.shape(Xin))
        num_rows, num_cols = Xin.shape

        # Compute the mean of each dimension, if required
        if(alg == 'LVQ'):
          if(not isQuery):
            X_means = np.mean(Xin, axis=0)
          else:
            X_means = X_means_p
          print("X_means shape:", np.shape(X_means))
        elif(alg == 'PDN' or alg == 'GMM'):
          X_means = None

        # Assign to X
        if(alg == 'LVQ'):
          # Center the input vectors
          X = Xin - X_means
        elif(alg == 'PDN' or alg == 'GMM'):
          X = Xin
        print("X shape:", np.shape(X))

        # Compute l, u
        if(alg == 'LVQ'): # LVQ: Always recompute l, u
          # Compute per-vector max and min across dimensions
          u = np.max(X, axis=1)
          l = np.min(X, axis=1)
        elif(alg == 'PDN'):
          if(not isQuery):
            # Compute per-dimension max and min across vectors
            u = np.max(X, axis=0)
            l = np.min(X, axis=0)
          else:
            # Reuse l
            l = l_p
            u = None
        elif(alg == 'GMM'):
          if(not isQuery):
            # Compute global max and min
            u = np.max(X)
            l = np.min(X)
          else:
            # Reuse l
            l = l_p
            u = None
        print("l shape:", np.shape(l))

        # Compute the quantization delta and its inverse
        if(alg == 'LVQ'): # Always recompute delta
          delta = (u - l) / (2**B1 - 1)
        elif(alg == 'PDN' or alg == 'GMM'):
          if(not isQuery):
            delta = (u - l) / (2**B1 - 1)
          else:   # Reuse delta_p
            delta = delta_p

        if(alg == 'LVQ' or alg == 'PDN'):
          delta[delta == 0] = 1.0  # Avoid division by zero
        elif(alg == 'GMM' and delta == 0):
          delta = 1   # Avoid division by zero

        inv_delta = 1.0 / delta
        print("delta shape:", np.shape(delta))
        print("inv_delta shape:", np.shape(inv_delta))

        # Compute the quantized codes
        if(alg == 'LVQ'):
          tcodes = np.floor(inv_delta[:, None] * (X - l[:, None]) + 0.5)
        elif(alg == 'PDN' or alg == 'GMM'):
          tcodes = np.floor(inv_delta[None, :] * (X - l[None, :]) + 0.5)
        tcodes = np.clip(tcodes, 0, 2**B1 - 1)

        # Convert the codes into integer format
        codes = tcodes.astype(int)

        # Shift to {-128...127} if required
        if(qdtype == 'SB1'):
          codes -= 128

        # Compute the first-level reconstruction "Xcomp"
        if(qdtype == 'UB1'):
          temp_codes = codes
        elif(qdtype == 'SB1'):
          temp_codes = codes+128

        if(alg == 'LVQ'):
          Xcomp = delta[:, None] * codes + l[:, None]
        elif(alg == 'PDN' or alg == 'GMM'):
          Xcomp = delta[None, :] * codes + l[None, :]

        # Add back the mean, if you had de-meaned
        if(alg == 'LVQ'):
          Xcomp += X_means

        print("Xcomp shape:", np.shape(Xcomp))
        print("codes shape:", np.shape(codes))

        return Xcomp, delta, codes, l, X_means

    @staticmethod
    def euclidean_distance(vector1, vector2):
        if len(vector1) != len(vector2):
            raise ValueError("Vectors must have the same length.")
        return np.sqrt(np.sum((np.array(vector1) - np.array(vector2)) ** 2))

    @staticmethod
    def cosine_distance(vector1, vector2):
        """Computes the cosine distance between two vectors."""
        dot_product = np.dot(vector1, vector2)
        norm_vec1 = np.linalg.norm(vector1)
        norm_vec2 = np.linalg.norm(vector2)

        if norm_vec1 == 0 or norm_vec2 == 0:
            return 1.0  # If either vector is zero, return max distance

        cosine_similarity = dot_product / (norm_vec1 * norm_vec2)
        return 1 - cosine_similarity  # Convert similarity to distance

    def distance(self, vector1, vector2):
      if(self.distfunc == 'EUCLIDEAN'):
        return self.euclidean_distance(vector1, vector2)
      elif(self.distfunc == 'COSINE'):
        return self.cosine_distance(vector1, vector2)

    # Compute reconstruction error
    @staticmethod
    def recon_error(original, reconstructed):
      if original.shape != reconstructed.shape:
        raise ValueError("Original and reconstructed vectors must have the same shape.")
      return np.mean(np.sqrt(np.sum((original - reconstructed) ** 2, axis=1)))

    @staticmethod
    def compute_accuracy(exact_knn, approx_knn):
      # if len(exact_knn) != len(approx_knn):
      #   raise ValueError("exact_knn and approx_knn must have the same length.")

      # exact_knn is of length K, approx_knn could be of length K*rescore_factor
      K = len(exact_knn)
      common = len(set(exact_knn).intersection(approx_knn))

      # Compute the accuracy as the fraction of correct neighbors
      accuracy = common / K
      return accuracy

# LVQ

In [ ]:
lvq = LVQFirstLevel(distfunc)

# Quantize database data
Xcomp, delta, codes, l_db, X_means_db = lvq.LVQ_first_level(Xin, B1, 'LVQ', False, qdtype)

print("Original Vectors:")
print(Xin)

print("\n8-Bit Quantized Vectors:")
print(codes)

print("\nAfter unquantizing from 8-bit format to float:")
print(Xcomp)

# Compute reconstruction error
recon_err = lvq.recon_error(Xin, Xcomp)

# Quantize query data
# Never used
Xcomp_query, delta_query, codes_query, l_q, X_means_q = lvq.LVQ_first_level(Xquery, B1, 'LVQ', True, qdtype, X_means_db, l_db, delta)

accuracies = []
accuracies_quant = []
for j, query_vector in enumerate (Xquery):
  if(j%50==0):
    print("\n================ j = ", j)
    print("\nAverage Quantized Accuracy for res_factor=", res_factors, ":  ", np.mean(np.asarray(accuracies_quant), axis=0))
  # Compute  distances and sort
  pqorig = PriorityQueue()
  pqcode = PriorityQueue()
  # pqxcomp = PriorityQueue()

  for i, x in enumerate(Xin):
      original_dist = lvq.distance(x, query_vector)
      code_dist = lvq.distance(codes[i], codes_query[j])
      # In our final code, we will use the distance between original query vector and unquantized graph vector
      # xcomp_dist = lvq.distance(Xcomp[i], query_vector)

      pqorig.put((original_dist, i))
      pqcode.put((code_dist, i))
      # pqxcomp.put((xcomp_dist, i))

  # print("\nOriginal TopK Vector Distances:")
  exact_knn = []
  exact_dists = []
  for _ in range(K):
      dist, idx = pqorig.get()
      # print(f"Vector ID: {idx}, Distance: {dist}")
      exact_knn.append(idx)
      exact_dists.append(dist)

  # print("\nTopK Distances with 8-bit quantized vectors:")
  quantized_knn = []
  for _ in range(len(res_factors)):
    quantized_knn.append([])

  quantized_dists = []
  for _ in range(len(res_factors)):
    quantized_dists.append([])

  for q in range(K*max(res_factors)):
      dist, idx = pqcode.get()
      # print(f"Vector ID: {idx}, Distance: {dist}")
      for w in range(len(res_factors)):
        if q < K*res_factors[w]:
          quantized_knn[w].append(idx)
          quantized_dists[w].append(dist)

  # approx_knn = []
  # # print("\nTopK Distances after unquantizing:")
  # for _ in range(K):
  #     dist, idx = pqxcomp.get()
  #     # print(f"Vector ID: {idx}, Distance: {dist}")
  #     approx_knn.append(idx)

  # accuracy = lvq.compute_accuracy(exact_knn, approx_knn)
  # accuracies.append(accuracy)
  # # print("\nAccuracy:", accuracy)

  accuracy_quant = [lvq.compute_accuracy(exact_knn, quantized_knn[i]) for i in range(len(res_factors))]
  accuracies_quant.append(accuracy_quant)
  # print(exact_knn)
  # print(exact_dists)
  # print(quantized_knn[0])
  # print(quantized_dists[0])
  # print(quantized_knn[1])
  # print(quantized_dists[1])
  # print(quantized_knn[2])
  # print(quantized_dists[2])
  print("Accuracy:", accuracy_quant, "\n")

print("\nAverage Reconstruction Error (RMS):", recon_err)
# print("\nAccuracies :", accuracies)
# print("\nAverage Unquantized Accuracy:", np.mean(accuracies))
print("\nAverage Quantized Accuracy for res_factor=", res_factors, ":  ", np.mean(np.asarray(accuracies_quant), axis=0))

Xin shape: (100000, 200)
X_means shape: (200,)
X shape: (100000, 200)
l shape: (100000,)
delta shape: (100000,)
inv_delta shape: (100000,)
Xcomp shape: (100000, 200)
codes shape: (100000, 200)
Original Vectors:
[[-0.23010001  0.07255     0.24698    ... -0.29379001 -0.20446999
  -0.23791   ]
 [-0.10979    -1.07290006  0.69889998 ... -0.21582    -0.48061001
  -0.11557   ]
 [ 0.075252    0.59464002  0.24169999 ... -0.078789    1.01730001
  -0.26394001]
 ...
 [ 0.51151001 -0.54062003 -0.1815     ...  0.59437001  0.5273
  -0.38343   ]
 [-0.22105999  0.12649     0.40748999 ...  0.31575    -0.17274
  -0.75369   ]
 [ 0.045832   -0.61195999  0.25566    ...  0.41709    -0.48008999
  -0.75964999]]

8-Bit Quantized Vectors:
[[-26  15  44 ... -77 -52   2]
 [ 26 -64  90 ...  -4 -22  40]
 [-10  31  -2 ... -45  59 -25]
 ...
 [ 47 -60 -26 ...  33  32 -22]
 [ -8  25  57 ...  28 -23 -51]
 [ 22 -61  37 ...  37 -58 -51]]

After unquantizing from 8-bit format to float:
[[-0.9729153  -0.66917285 -0.4930426  

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Accuracy: [0.5, 0.8, 1.0] 

Accuracy: [0.4, 0.7, 1.0] 

Accuracy: [0.0, 0.0, 0.1] 

Accuracy: [0.5, 0.9, 1.0] 

Accuracy: [0.0, 0.0, 0.0] 

Accuracy: [0.8, 1.0, 1.0] 

Accuracy: [0.5, 0.6, 0.8] 

Accuracy: [0.3, 0.8, 1.0] 

Accuracy: [0.0, 0.0, 0.1] 

Accuracy: [0.5, 1.0, 1.0] 

Accuracy: [0.7, 1.0, 1.0] 

Accuracy: [0.3, 0.7, 0.9] 

Accuracy: [0.3, 0.6, 1.0] 

Accuracy: [0.5, 0.9, 1.0] 

Accuracy: [0.6, 0.9, 0.9] 

Accuracy: [0.2, 0.5, 0.7] 

Accuracy: [0.7, 0.9, 1.0] 

Accuracy: [0.7, 1.0, 1.0] 

Accuracy: [0.3, 0.9, 0.9] 

Accuracy: [0.5, 0.9, 1.0] 

Accuracy: [0.3, 0.7, 0.9] 

Accuracy: [0.2, 0.5, 1.0] 

Accuracy: [0.0, 0.0, 0.1] 

Accuracy: [0.7, 1.0, 1.0] 

Accuracy: [0.0, 0.6, 0.6] 

Accuracy: [0.5, 1.0, 1.0] 

Accuracy: [0.6, 1.0, 1.0] 

Accuracy: [0.9, 1.0, 1.0] 

Accuracy: [0.8, 1.0, 1.0] 

Accuracy: [0.2, 0.4, 0.7] 

Accuracy: [0.3, 0.5, 0.5] 

Accuracy: [0.4, 0.7, 1.0] 

Accuracy: [0.6, 0.9, 1.0] 

Accuracy: [0.4, 0.9, 1.0] 

Accuracy: [0.2, 0.3, 0.4] 

Accuracy: [0.6, 0.9,

# Per Dimension Norm

In [ ]:
lvq = LVQFirstLevel(distfunc)

# Quantize database data
Xcomp, delta, codes, l_db, X_means_db = lvq.LVQ_first_level(Xin, B1, alg, False, qdtype)

print(np.min(codes))
print(np.max(codes))

print("Original Vectors:")
print(Xin)

print("\n8-Bit Quantized Vectors:")
print(codes)

print("\nAfter unquantizing from 8-bit format to float:")
print(Xcomp)

# Compute reconstruction error
recon_err = lvq.recon_error(Xin, Xcomp)

# Quantize query data
# Never used
Xcomp_query, delta_query, codes_query, l_q, X_means_q = lvq.LVQ_first_level(Xquery, B1, alg, True, qdtype, X_means_db, l_db, delta)

accuracies = []
accuracies_quant = []
for j, query_vector in enumerate (Xquery):
  if(j%50==0):
    print("\n================ j = ", j)
    print("\nAverage Quantized Accuracy for res_factor=", res_factors, ":  ", np.mean(np.asarray(accuracies_quant), axis=0))
  # Compute  distances and sort
  pqorig = PriorityQueue()
  pqcode = PriorityQueue()
  # pqxcomp = PriorityQueue()

  for i, x in enumerate(Xin):
      original_dist = lvq.distance(x, query_vector)
      code_dist = lvq.distance(codes[i], codes_query[j])
      # In our final code, we will use the distance between original query vector and unquantized graph vector
      # xcomp_dist = lvq.distance(Xcomp[i], query_vector)

      pqorig.put((original_dist, i))
      pqcode.put((code_dist, i))
      # pqxcomp.put((xcomp_dist, i))

  # print("\nOriginal TopK Vector Distances:")
  exact_knn = []
  exact_dists = []
  for _ in range(K):
      dist, idx = pqorig.get()
      # print(f"Vector ID: {idx}, Distance: {dist}")
      exact_knn.append(idx)
      exact_dists.append(dist)

  # print("\nTopK Distances with 8-bit quantized vectors:")
  quantized_knn = []
  for _ in range(len(res_factors)):
    quantized_knn.append([])

  quantized_dists = []
  for _ in range(len(res_factors)):
    quantized_dists.append([])

  for q in range(K*max(res_factors)):
      dist, idx = pqcode.get()
      # print(f"Vector ID: {idx}, Distance: {dist}")
      for w in range(len(res_factors)):
        if q < K*res_factors[w]:
          quantized_knn[w].append(idx)
          quantized_dists[w].append(dist)

  # approx_knn = []
  # # print("\nTopK Distances after unquantizing:")
  # for _ in range(K):
  #     dist, idx = pqxcomp.get()
  #     # print(f"Vector ID: {idx}, Distance: {dist}")
  #     approx_knn.append(idx)

  # accuracy = lvq.compute_accuracy(exact_knn, approx_knn)
  # accuracies.append(accuracy)
  # # print("\nAccuracy:", accuracy)

  accuracy_quant = [lvq.compute_accuracy(exact_knn, quantized_knn[i]) for i in range(len(res_factors))]
  accuracies_quant.append(accuracy_quant)
  print("Accuracy:", accuracy_quant, "\n")

print("\nAverage Reconstruction Error (RMS):", recon_err)
# print("\nAccuracies :", accuracies)
# print("\nAverage Unquantized Accuracy:", np.mean(accuracies))
print("\nAverage Quantized Accuracy for res_factor=", res_factors, ":  ", 100*np.mean(np.asarray(accuracies_quant), axis=0))

# Combined

In [ ]:
# Returns exact_knn: Shape (num_query_vectors, K)
def get_exact_topk(Xin, Xquery, lvq):
  # Number of dimensions must be the same
  assert(np.shape(Xin)[1] == np.shape(Xquery)[1])

  num_query_vectors = np.shape(Xquery)[0]

  print("Populate the priority queue for each query")
  pqorig = []
  for _ in range(num_query_vectors):
    pqorig.append(PriorityQueue())
  for j, query_vector in enumerate (Xquery):
    for i, x in enumerate(Xin):
      original_dist = lvq.euclidean_distance(x, query_vector)
      pqorig[j].put((original_dist, i))

  print("Populate the topK VIDs for each query")
  exact_knn = []
  for _ in range(num_query_vectors):
    exact_knn.append([])
  for j, query_vector in enumerate (Xquery):
    for _ in range(K):
      dist, idx = pqorig[j].get()
      # print(f"Vector ID: {idx}, Distance: {dist}")
      print(j, num_query_vectors)
      exact_knn[j].append(idx)

  return exact_knn

In [ ]:
compute_exact   = True
# compute algorithms
doThisAlg = [True, True]

num_algs = 2 # LVQ, PerDim
# Xcomp, delta, codes, l_db, X_means_db, Xcomp_query, delta_query, codes_query, l_q, X_means_q  = []
# # Accuracies with unquantized stored vectors
# accuracies = []     # LVQ, PerDim
# # Accuracies with quantized query vectors
# accuracies_quant = []     # LVQ, PerDim
# for _ in range(num_algs):
#   Xcomp.append([])
#   delta.append([])
#   codes.append([])
#   l_db.append([])
#   X_means_db.append([])

#   Xcomp_query.append([])
#   delta_query.append([])
#   codes_query.append([])
#   l_q.append([])
#   X_means_q.append([])

#   accuracies.append([])
#   accuracies_quant.append([])

lvq = LVQFirstLevel()


if(compute_exact or exact_knn.any() == None):
  # Compute the exact topK for all query vectors:
  exact_knn = get_exact_topk(Xin, Xquery, lvq)


for algidx in range(num_algs):
  if(doThisAlg[algidx]):
    isPerDim = (algidx == 1)

    # Quantize database data, LVQ
    Xcomp[algidx], delta[algidx], codes[algidx], l_db[algidx], \
      X_means_db[algidx] = \
      lvq.LVQ_first_level(Xin, B1, isPerDim, False)
    # Quantize query data, LVQ
    Xcomp_query[algidx], delta_query[algidx], codes_query[algidx], l_q[algidx],\
      X_means_q[algidx] = \
      lvq.LVQ_first_level(Xquery[algidx], B1, isPerDim, True, \
        X_means_db[algidx], l_db[algidx], delta[algidx])

    # print("Original Vectors:")
    # print(Xin)

    # print("\n8-Bit Quantized Vectors:")
    # print(codes[algidx])

    # print("\nAfter unquantizing from 8-bit format to float:")
    # print(Xcomp[algidx])


    # Compute reconstruction error
    recon_err = lvq.recon_error(Xin, Xcomp)


    for j, query_vector in enumerate (Xquery):
      if(j%50==0):
        print("\n================ j = ", j)
        print("\nAverage Quantized Accuracy for res_factor=", res_factors, ":  ", np.mean(np.asarray(accuracies_quant), axis=0))
      # Compute distances and sort
      pqcodes = []     # LVQ, PerDim
      pqxcomps = []     # LVQ, PerDim
      for _ in range(num_algs):
        pqcodes.append(PriorityQueue())
        pqxcomps.append(PriorityQueue())

      for i, x in enumerate(Xin):
        ## LVQ
        code_dist = lvq.distance(codes[i], codes_query[j])
        # xcomp_dist = lvq.euclidean_distance(Xcomp[i], query_vector)

        pqcodes[0].put((code_dist, i))
        # pqxcomp[0].put((xcomp_dist, i))

        ## PerDim
        code_dist = lvq.distance(codes_2[i], codes_query[j])
        # xcomp_dist = lvq.euclidean_distance(Xcomp_2[i], query_vector)

        pqcodes[1].put((code_dist, i))
        # pqxcomp[1].put((xcomp_dist, i))

      ## Quantized Accuracy
      # print("\nTopK Distances with 8-bit quantized vectors:")
      for e in num_algs: # LVQ, PerDim
        quantized_knn = []
        for _ in range(len(res_factors)):
          quantized_knn.append([])

        for q in range(K*max(res_factors)):
            dist, idx = pqcodes[e].get()
            # print(f"Vector ID: {idx}, Distance: {dist}")
            for w in range(len(res_factors)):
              if q < K*res_factors[w]:
                quantized_knn[w].append(idx)

        accuracy_quant = [lvq.compute_accuracy(exact_knn[j], quantized_knn[w]) for w in range(len(res_factors))]
        accuracies_quant[e].append(accuracy_quant)
        print("\nAccuracy:", accuracy_quant)

      ## Unquantized Accuracy
      # for e in num_algs: # LVQ, PerDim
        # approx_knn = []
        # # print("\nTopK Distances after unquantizing:")
        # for _ in range(K):
        #     dist, idx = pqxcomps[e].get()
        #     # print(f"Vector ID: {idx}, Distance: {dist}")
        #     approx_knn.append(idx)

        # accuracy = lvq.compute_accuracy(exact_knn[j], approx_knn)
        # accuracies[e].append(accuracy)
        # print("\nAccuracy:", accuracy)

print("\nAverage Reconstruction Error (RMS):", recon_err)
for e in num_algs: # LVQ, PerDim
  # print("\nAccuracies :", accuracies[e])
  # print("\nAverage Unquantized Accuracy:", np.mean(accuracies[e]))
  print("\nAverage Quantized Accuracy for algorithm", e, " for res_factor=", res_factors, ":  ", np.mean(np.asarray(accuracies_quant[e]), axis=0))

Xin shape: (10000, 960)
X_means shape: (960,)
X shape: (10000, 960)
l,u shape: (10000,)
delta shape: (10000,)
inv_delta shape: (10000,)
Xcomp shape: (10000, 960)
codes shape: (10000, 960)
Xin shape: (1000, 960)
X_means shape: (960,)
X shape: (1000, 960)
l,u shape: (1000,)
delta shape: (1000,)
inv_delta shape: (1000,)
Xcomp shape: (1000, 960)
codes shape: (1000, 960)
Xin shape: (10000, 960)
X_means shape: (960,)
X shape: (10000, 960)
l,u shape: (960,)
delta shape: (960,)
inv_delta shape: (960,)
Xcomp shape: (10000, 960)
codes shape: (10000, 960)
Xin shape: (1000, 960)
X_means shape: (960,)
X shape: (1000, 960)
l,u shape: (960,)
delta shape: (960,)
inv_delta shape: (960,)
Xcomp shape: (1000, 960)
codes shape: (1000, 960)
Populate the priority queue for each query
Populate the topK VIDs for each query
0 1000


IndexError: list index out of range